# Pacotes usados

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats


In [2]:
%pwd

'/home/gilberto/Insync/sassi.pereira.gilberto@gmail.com/Google Drive - Shared with me/Funarte 2024/1. Pesquisa/3. Metodologia/analise_respostas/scripts'

In [3]:
%cd ..

/home/gilberto/Insync/sassi.pereira.gilberto@gmail.com/Google Drive - Shared with me/Funarte 2024/1. Pesquisa/3. Metodologia/analise_respostas


In [4]:
%pwd

'/home/gilberto/Insync/sassi.pereira.gilberto@gmail.com/Google Drive - Shared with me/Funarte 2024/1. Pesquisa/3. Metodologia/analise_respostas'

# Lendo os dados

In [5]:
dados = pd.read_excel('dados/brutos/respostas.xlsx', sheet_name='dados_2024_12_12')
dados.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 72 entries, 0 to 71
Columns: 161 entries, tipo_ente to Unnamed: 160
dtypes: float64(2), int64(1), object(158)
memory usage: 90.7+ KB


In [6]:
dados = dados[dados.eh_capital == "Estado"]
dados = dados.reset_index(drop=True)

In [7]:
dados.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27 entries, 0 to 26
Columns: 161 entries, tipo_ente to Unnamed: 160
dtypes: float64(2), int64(1), object(158)
memory usage: 34.1+ KB


In [8]:
dados.columns

Index(['tipo_ente', 'eh_capital', 'identificacao', 'nome_orgao', 'dirigente',
       'website', 'tel_orgao', 'nome_respondente', 'tel_respondente',
       'email_respondente',
       ...
       'atvs_previstas_editais_lpg___6', 'atvs_previstas_editais_lpg___7',
       'atvs_previstas_editais_lpg___8', 'atvs_previstas_editais_lpg___9',
       'atvs_previstas_editais_lpg___10', 'atvs_previstas_editais_lpg___11',
       'atvs_previstas_editais_lpg___12', 'atvs_previstas_editais_lpg_outras',
       'pesquisa_fomento_s_artes_no_brasil_complete', 'Unnamed: 160'],
      dtype='object', length=161)

In [9]:
def tabela_qualitativa(dados, variavel, destino = 'tabelas'):
    """
    Calcula a tabela de distribuição de frequências para uma variável qualitativa.

    args:
        dados: data frame
        variavel: coluna do data frame (variável qualitativa)
        destino: pasta para salvar a tabela. Valor padrão é 'tabelas'

    return:
        Tabela de distribuição de frequências como pd.DataFrame
    """
    tabela = pd.DataFrame({
        'frequencia': dados[variavel].value_counts(),
        'porcentagem': dados[variavel].value_counts(normalize=True)
    })
    tabela.loc['Total'] = tabela.sum()
    tabela.reset_index(inplace=True)
    tabela.to_excel(f'{destino}/{variavel}.xlsx', index=False)
    return tabela

In [10]:
def tabela_checkbox(dados, variavel, k, rotulos = [],  destino = 'tabelas'):
    """
    Calcula a tabela de distribuição de frequêncis para uma pergunta do tipo checkbox.

    args:
        dados: data frame
        variavel: nome da variavel
        k: número de categorias
        destino: pasta para salvar a tabela. Valor padrão é 'tabelas'

    return:
        Tabela de distribuiação de frequências como pd.DataFrame
    """
    categorias = [f'{variavel}___' + str(i) for i in range(1, k+1)]
    tabela = pd.DataFrame({
        'colunas': categorias
    })
    if bool(rotulos):
        tabela['rotulos'] = rotulos
    tabela['frequencia'] = [np.sum(dados[categoria] == 'Checked') for categoria in categorias]
    tabela['porcentagem'] = tabela['frequencia'] / dados.shape[0]
    tabela.to_excel(f'{destino}/{variavel}.xlsx', index=False)
    return tabela

In [11]:
def contagem_coluna_linkert(dados, coluna):
    """
    Calcula a frequencia de cada um dos valores da lista [1, 2, 3, 4, 5, 'Não se aplica']

    args:
        dados: data frame
        coluna: nome da coluna com letras das perguntas

    return:
        Lista com a contagem das letras das perguntas.
    """
    opcoes = [1, 2, 3, 4, 5, 'Não se aplica']
    contagem = dados[coluna].value_counts()
    return [contagem.loc[i] if i in contagem.index else 0 for i in opcoes]

In [17]:
def tabela_linkert(dados, nome, categorias, destino='tabelas'):
    """
    Calcula a tabela de distribuição de frequências as perguntas em escala linkert.

    args:
        dados: datda frame
        nome: nome da variavel
        categorias: letras das escalas linkert
        destino: pasta para salvar a tabela. Valor padrão é 'tabelas'

    return:
        Tabela de distribuição de frequências como pd.DataFrame
    """
    categorias = [f'{nome}_{categoria}' for categoria in categorias]
    dicio = {categoria: contagem_coluna_linkert(dados, categoria) for categoria in categorias}
    tabela = pd.DataFrame(dicio, index = [1, 2, 3, 4, 5, 'Não se aplica'])
    tabela.T.to_excel(f'{destino}/{nome}.xlsx')
    return tabela.T


In [13]:
quantil = lambda x, q: np.quantile(x.dropna(), q, axis=0)

In [14]:
def describe_pt(dados, nome, destino = 'tabelas'):
    """
    Método describe em português.

    args:
        dados: data frame
        nome: nome da variável quantitativa

    return:
        Data frame com as medidas de resumo
    """
    sumario = dados[nome].agg({
        'Média': 'mean', 'Mediana': 'median', 'Primeiro quartil': lambda x: quantil(x, 0.25),
        'Terceiro quartil': lambda x: quantil(x, 3/4), 'Mínimo': 'min', 'Máximo': 'max',
        'Valores não missing': 'count', 'Variância': 'var', 
        'Desvio padrão': 'std'
    }).to_frame()
    sumario.to_excel(f'{destino}/{nome}.xlsx')
    return sumario

# Tabelas

Estou usando os nomes e padrões do dicionário de dados: https://docs.google.com/spreadsheets/d/1vb2YsNy49XFDDSj5PTpIHP8YJOtXjSRjb2VTVrxoorc/edit?usp=sharing.

## `tipo_ente`

In [118]:
tabela_qualitativa(dados, 'tipo_dep')

,tipo_dep,frequencia,porcentagem
0,Secretaria,13.0,0.481481
1,Fundação,4.0,0.148148
2,Outro(s). Especifique: {tipo_dep_outros},3.0,0.111111
3,Superintendência,2.0,0.074074
4,Subsecretaria,2.0,0.074074
5,Coordenadoria,1.0,0.037037
6,Diretoria,1.0,0.037037
7,Gerência,1.0,0.037037
8,Total,27.0,1.000000


In [119]:
np.sum(dados.tipo_dep.value_counts())

27

## `linguagens_orgao` e `linguagens_outros`

In [120]:
rotulos = [
    "Artes Visuais",
    "Audiovisual",
    "Circo",
    "Dança",
    "Literatura",
    "Música",
    "Teatro",
    "Outra(s)."
]

In [121]:
tabela_checkbox(dados, 'linguagens_orgao', 8, rotulos=rotulos)

,colunas,rotulos,frequencia,porcentagem
0,linguagens_orgao___1,Artes Visuais,25,0.925926
1,linguagens_orgao___2,Audiovisual,25,0.925926
2,linguagens_orgao___3,Circo,25,0.925926
3,linguagens_orgao___4,Dança,26,0.962963
4,linguagens_orgao___5,Literatura,26,0.962963
5,linguagens_orgao___6,Música,27,1.000000
6,linguagens_orgao___7,Teatro,27,1.000000
7,linguagens_orgao___8,Outra(s).,14,0.518519


## `processos_decisorios`

In [18]:
tabela_linkert(dados, 'processos_decisorios', ['a', 'b', 'c', 'd', 'e', 'outras'])

,1,2,3,4,5,Não se aplica
processos_decisorios_a,1,1,5,9,10,1
processos_decisorios_b,3,2,9,6,7,0
processos_decisorios_c,10,7,1,0,2,7
processos_decisorios_d,0,1,3,9,12,2
processos_decisorios_e,13,5,2,0,0,7
processos_decisorios_outras,3,1,0,0,1,22


In [123]:
list(dados['processos_decisorios_a'].value_counts().index)

[5, 4, 3, 'Não se aplica', 2, 1]

## `plano_cultura`

In [124]:
tabela_qualitativa(dados, 'plano_cultura')

,plano_cultura,frequencia,porcentagem
0,Sim,18.0,0.666667
1,Em elaboração,7.0,0.259259
2,Não,2.0,0.074074
3,Total,27.0,1.000000


## `planos_setoriais`

In [125]:
tabela_qualitativa(dados, 'planos_setoriais')

,planos_setoriais,frequencia,porcentagem
0,Não,12.0,0.444444
1,Em elaboração,11.0,0.407407
2,Sim,4.0,0.148148
3,Total,27.0,1.000000


## `desafios_implementacao`

In [126]:
rotulos = [
    "Ausência de indicadores de avaliação eficazes",
    "Burocracia excessiva",
    "Carência de formação e qualificação profissional",
    "Dificuldade em realizar processos de consulta e participação da sociedade",
    "Estrutura deficiente do órgão",
    "Falhas na comunicação entre o ente e a sociedade civil",
    "Financiamento insuficiente para a implementação da política",
    "Influência política no processo decisório",
    "Limitações na articulação e apoio dos órgãos de controle e outras instâncias decisórias",
    "Não foram identificados desafios",
    "Outro(s)."
]

In [127]:
tabela_checkbox(dados, 'desafios_implementacao', 11, rotulos=rotulos)

,colunas,rotulos,frequencia,porcentagem
0,desafios_implementacao___1,Ausência de indicadores de avaliação eficazes,18,0.666667
1,desafios_implementacao___2,Burocracia excessiva,9,0.333333
2,desafios_implementacao___3,Carência de formação e qualificação profissional,9,0.333333
3,desafios_implementacao___4,Dificuldade em realizar processos de consulta ...,2,0.074074
4,desafios_implementacao___5,Estrutura deficiente do órgão,13,0.481481
5,desafios_implementacao___6,Falhas na comunicação entre o ente e a socieda...,2,0.074074
6,desafios_implementacao___7,Financiamento insuficiente para a implementaçã...,11,0.407407
7,desafios_implementacao___8,Influência política no processo decisório,2,0.074074
8,desafios_implementacao___9,Limitações na articulação e apoio dos órgãos d...,5,0.185185
9,desafios_implementacao___10,Não foram identificados desafios,0,0.000000


## `fonte_utilizadas`

In [128]:
rotulos = [
    "Captação de recursos complementares",
    "Captação de recursos privados, com ou sem incentivo fiscal",
    "Dotações orçamentárias",
    "Fundos públicos destinados às políticas públicas culturais",
    "Outras fontes ou mecanismos previstos em legislação específica",
    "Rendimentos obtidos durante a execução da ação cultural",
    "Outra(s)."
]

In [129]:
tabela_checkbox(dados, 'fontes_utilizadas', 7, rotulos=rotulos)

,colunas,rotulos,frequencia,porcentagem
0,fontes_utilizadas___1,Captação de recursos complementares,12,0.444444
1,fontes_utilizadas___2,"Captação de recursos privados, com ou sem ince...",15,0.555556
2,fontes_utilizadas___3,Dotações orçamentárias,22,0.814815
3,fontes_utilizadas___4,Fundos públicos destinados às políticas públic...,21,0.777778
4,fontes_utilizadas___5,Outras fontes ou mecanismos previstos em legis...,16,0.592593
5,fontes_utilizadas___6,Rendimentos obtidos durante a execução da ação...,11,0.407407
6,fontes_utilizadas___7,Outra(s).,7,0.259259


## `modalidades_utilizadas`

In [130]:
rotulos = [
    "Apoio a espaços culturais",
    "Apoio à execução de ações culturais",
    "Bolsas culturais",
    "Cooperação cultural",
    "Ocupação cultural",
    "Premiação cultural",
    "Outra(s)."
]

In [131]:
tabela_checkbox(dados, 'modalidades_utilizadas', 7, rotulos=rotulos)

,colunas,rotulos,frequencia,porcentagem
0,modalidades_utilizadas___1,Apoio a espaços culturais,20,0.740741
1,modalidades_utilizadas___2,Apoio à execução de ações culturais,27,1.000000
2,modalidades_utilizadas___3,Bolsas culturais,8,0.296296
3,modalidades_utilizadas___4,Cooperação cultural,14,0.518519
4,modalidades_utilizadas___5,Ocupação cultural,10,0.370370
5,modalidades_utilizadas___6,Premiação cultural,27,1.000000
6,modalidades_utilizadas___7,Outra(s).,5,0.185185


## `editais_especificos_2023`

In [132]:
tabela_qualitativa(dados, 'editais_especificos_2023')

,editais_especificos_2023,frequencia,porcentagem
0,Sim,22.0,0.814815
1,Não,5.0,0.185185
2,Total,27.0,1.000000


## `como_avaliacao_politica`

In [133]:
rotulos = [
    "Análise dos resultados dos programas, ações e chamadas públicas",
    "O órgão não realiza avaliação da política para as artes",
    "Realização de pesquisas",
    "Reuniões periódicas com o setor (participação social)",
    "Não existe política para as artes",
    "Outra(s)."
]

In [134]:
tabela_checkbox(dados, 'como_avaliacao_politica', 6, rotulos=rotulos)

,colunas,rotulos,frequencia,porcentagem
0,como_avaliacao_politica___1,"Análise dos resultados dos programas, ações e ...",21,0.777778
1,como_avaliacao_politica___2,O órgão não realiza avaliação da política para...,2,0.074074
2,como_avaliacao_politica___3,Realização de pesquisas,8,0.296296
3,como_avaliacao_politica___4,Reuniões periódicas com o setor (participação ...,16,0.592593
4,como_avaliacao_politica___5,Não existe política para as artes,1,0.037037
5,como_avaliacao_politica___6,Outra(s).,3,0.111111


## `quem_executa`

In [135]:
rotulos = [
    "Conselho, Fóruns e outros (participação social)",
    "Empresa contratada",
    "Observatório ou outra instituição pública externa ao organismo",
    "Setor(es) do próprio organismo",
    "Outra(s)."
]

In [136]:
tabela_checkbox(dados, 'quem_executa', 5, rotulos)

,colunas,rotulos,frequencia,porcentagem
0,quem_executa___1,"Conselho, Fóruns e outros (participação social)",16,0.592593
1,quem_executa___2,Empresa contratada,2,0.074074
2,quem_executa___3,Observatório ou outra instituição pública exte...,6,0.222222
3,quem_executa___4,Setor(es) do próprio organismo,19,0.703704
4,quem_executa___5,Outra(s).,1,0.037037


## `monitoramento_resultados`

In [137]:
tabela_qualitativa(dados, 'monitoramento_resultados')

,monitoramento_resultados,frequencia,porcentagem
0,Sim,15.0,0.555556
1,Não,9.0,0.333333
2,Em implementação,3.0,0.111111
3,Total,27.0,1.000000


## `valor_complementar_lab_numero`

In [138]:
describe_pt(dados, 'valor_complementar_lab_numero')

,valor_complementar_lab_numero
Média,9.924299e+06
Mediana,3.000000e+06
Primeiro quartil,1.397310e+06
Terceiro quartil,1.947876e+07
Mínimo,1.404000e+05
Máximo,2.457755e+07
Valores não missing,7.000000e+00
Variância,1.144976e+14
Desvio padrão,1.070035e+07


## `how_def_valores_lab`

In [19]:
tabela_linkert(dados, 'how_def_valores_lab', ['a', 'b', 'c', 'd', 'e', 'f'])

,1,2,3,4,5,Não se aplica
how_def_valores_lab_a,1,2,3,7,10,4
how_def_valores_lab_b,2,4,9,3,8,1
how_def_valores_lab_c,7,4,3,2,2,9
how_def_valores_lab_d,1,0,3,6,17,0
how_def_valores_lab_e,14,3,1,0,1,8
how_def_valores_lab_f,1,0,0,0,3,23


## `modalidades_fomento`

In [140]:
rotulos = [
    "Apoio a espaços culturais",
    "Apoio à execução de ações culturais",
    "Bolsas culturais",
    "Cooperação cultural",
    "Ocupação cultural",
    "Premiação cultural",
    "Outra(s)."
]

In [141]:
tabela_checkbox(dados, 'modalidades_fomento', 7, rotulos)

,colunas,rotulos,frequencia,porcentagem
0,modalidades_fomento___1,Apoio a espaços culturais,15,0.555556
1,modalidades_fomento___2,Apoio à execução de ações culturais,23,0.851852
2,modalidades_fomento___3,Bolsas culturais,6,0.222222
3,modalidades_fomento___4,Cooperação cultural,4,0.148148
4,modalidades_fomento___5,Ocupação cultural,5,0.185185
5,modalidades_fomento___6,Premiação cultural,26,0.962963
6,modalidades_fomento___7,Outra(s).,2,0.074074


## `elos_contemplados_lab`

In [142]:
rotulos = [
    "Acesso",
    "Criação",
    "Difusão",
    "Formação",
    "Memória",
    "Pesquisa"
]

In [143]:
tabela_checkbox(dados, 'elos_contemplados_lab', 6, rotulos)

,colunas,rotulos,frequencia,porcentagem
0,elos_contemplados_lab___1,Acesso,18,0.666667
1,elos_contemplados_lab___2,Criação,26,0.962963
2,elos_contemplados_lab___3,Difusão,27,1.000000
3,elos_contemplados_lab___4,Formação,25,0.925926
4,elos_contemplados_lab___5,Memória,21,0.777778
5,elos_contemplados_lab___6,Pesquisa,17,0.629630


## `atvs_previstas_lab`

In [144]:
rotulos = [
    "Apresentação",
    "Atividades de mediação",
    "Atividades de reflexão",
    "Circulação",
    "Exibição",
    "Intercâmbio",
    "Plano de atividades - manutenção de espaços",
    "Plano de atividades - manutenção de grupos e coletivos",
    "Preservação de acervos",
    "Realização de eventos calendarizados",
    "Residência",
    "Outra(s)."
]

In [145]:
tabela_checkbox(dados, 'atvs_previstas_lab', 12, rotulos)

,colunas,rotulos,frequencia,porcentagem
0,atvs_previstas_lab___1,Apresentação,24,0.888889
1,atvs_previstas_lab___2,Atividades de mediação,10,0.370370
2,atvs_previstas_lab___3,Atividades de reflexão,13,0.481481
3,atvs_previstas_lab___4,Circulação,21,0.777778
4,atvs_previstas_lab___5,Exibição,23,0.851852
5,atvs_previstas_lab___6,Intercâmbio,9,0.333333
6,atvs_previstas_lab___7,Plano de atividades - manutenção de espaços,11,0.407407
7,atvs_previstas_lab___8,Plano de atividades - manutenção de grupos e c...,11,0.407407
8,atvs_previstas_lab___9,Preservação de acervos,11,0.407407
9,atvs_previstas_lab___10,Realização de eventos calendarizados,14,0.518519


## `valor_complementar_lpg_numero`

In [146]:
describe_pt(dados, 'valor_complementar_lpg_numero')

,valor_complementar_lpg_numero
Média,1.897898e+07
Mediana,1.224200e+07
Primeiro quartil,7.462415e+06
Terceiro quartil,2.712706e+07
Mínimo,2.682831e+06
Máximo,4.201212e+07
Valores não missing,3.000000e+00
Variância,4.207385e+14
Desvio padrão,2.051191e+07


## `def_atvs_valores_lpg`

In [20]:
tabela_linkert(dados, 'def_atvs_valores_lpg', ['a', 'b', 'c', 'd', 'e', 'f'])

,1,2,3,4,5,Não se aplica
def_atvs_valores_lpg_a,1,0,4,8,14,0
def_atvs_valores_lpg_b,1,0,6,9,8,3
def_atvs_valores_lpg_c,10,3,2,0,2,10
def_atvs_valores_lpg_d,1,1,1,5,17,2
def_atvs_valores_lpg_e,14,3,0,0,0,10
def_atvs_valores_lpg_f,2,0,0,0,1,24


## `modalidades_lpg`

In [148]:
rotulos = [
    "Apoio a espaços culturais",
    "Apoio à execução de ações culturais",
    "Bolsas culturais",
    "Cooperação cultural",
    "Ocupação cultural",
    "Premiação cultural",
    "Outra(s)."
]

In [149]:
tabela_checkbox(dados, 'modalidades_lpg', 7, rotulos)

,colunas,rotulos,frequencia,porcentagem
0,modalidades_lpg___1,Apoio a espaços culturais,20,0.740741
1,modalidades_lpg___2,Apoio à execução de ações culturais,27,1.000000
2,modalidades_lpg___3,Bolsas culturais,9,0.333333
3,modalidades_lpg___4,Cooperação cultural,5,0.185185
4,modalidades_lpg___5,Ocupação cultural,6,0.222222
5,modalidades_lpg___6,Premiação cultural,23,0.851852
6,modalidades_lpg___7,Outra(s).,1,0.037037


## `elos_contemplados_lpg`

In [150]:
rotulos = [
    "Acesso",
    "Criação",
    "Difusão",
    "Formação",
    "Memória",
    "Pesquisa"
]

In [151]:
tabela_checkbox(dados, 'elos_contemplados_lpg', 6, rotulos)

,colunas,rotulos,frequencia,porcentagem
0,elos_contemplados_lpg___1,Acesso,24,0.888889
1,elos_contemplados_lpg___2,Criação,27,1.000000
2,elos_contemplados_lpg___3,Difusão,27,1.000000
3,elos_contemplados_lpg___4,Formação,26,0.962963
4,elos_contemplados_lpg___5,Memória,22,0.814815
5,elos_contemplados_lpg___6,Pesquisa,20,0.740741


## `atvs_previstas_editais_lpg`

In [152]:
rotulos = [
    "Apresentação",
    "Atividades de mediação",
    "Atividades de reflexão",
    "Circulação",
    "Exibição",
    "Intercâmbio",
    "Plano de mtividades - manutenção de espaços",
    "Plano de atividades - manutenção de grupos e coletivos",
    "Preservação de acervos",
    "Realização de eventos calendarizados",
    "Residência",
    "Outra(s)."
]

In [153]:
tabela_checkbox(dados, 'atvs_previstas_editais_lpg', 12, rotulos)

,colunas,rotulos,frequencia,porcentagem
0,atvs_previstas_editais_lpg___1,Apresentação,26,0.962963
1,atvs_previstas_editais_lpg___2,Atividades de mediação,17,0.629630
2,atvs_previstas_editais_lpg___3,Atividades de reflexão,11,0.407407
3,atvs_previstas_editais_lpg___4,Circulação,25,0.925926
4,atvs_previstas_editais_lpg___5,Exibição,26,0.962963
5,atvs_previstas_editais_lpg___6,Intercâmbio,9,0.333333
6,atvs_previstas_editais_lpg___7,Plano de mtividades - manutenção de espaços,15,0.555556
7,atvs_previstas_editais_lpg___8,Plano de atividades - manutenção de grupos e c...,10,0.370370
8,atvs_previstas_editais_lpg___9,Preservação de acervos,15,0.555556
9,atvs_previstas_editais_lpg___10,Realização de eventos calendarizados,18,0.666667
